In [ ]:
!pip install openai

In [ ]:
import pandas as pd

In [ ]:
# initialize openai api 
import openai
openai.api_key = OPENAI_API_KEY

In [ ]:
# this example below will be replaced by user input 
user_input_ingredients = ["butter", "salt", "bacon", "milk", "flour"]
user_input_diet_type = 'keto'
user_input_allergens = 'gluten'

In [ ]:
# predefined examples for fewshot
new_fewshot_examples_restrictions = [{'title': 'No-Bake Nut Cookies',
  'ingredients': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]',
  'directions': '["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]',
  'diet_type': 'vegeterian',
  'allergy_type': 'tree nuts, dairy'},
 {'title': "Jewell Ball'S Chicken",
  'ingredients': '["beef", "chicken breasts", "cream of mushroom soup", "sour cream"]',
  'directions': '["Place chipped beef on bottom of baking dish.", "Place chicken on top of beef.", "Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\\u00b0 for 3 hours."]',
  'diet_type': 'none',
  'allergy_type': 'none'},
 {'title': 'Creamy Corn',
  'ingredients': '["frozen corn", "cream cheese", "butter", "garlic powder", "salt", "pepper"]',
  'directions': '["In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings."]',
  'diet_type': 'vegetarian',
  'allergy_type': 'none'},
 {'title': 'Chicken Funny',
  'ingredients': '["chicken", "chicken gravy", "cream of mushroom soup", "shredded cheese"]',
  'directions': '["Boil and debone chicken.", "Put bite size pieces in average size square casserole dish.", "Pour gravy and cream of mushroom soup over chicken; level.", "Make stuffing according to instructions on box (do not make too moist).", "Put stuffing on top of chicken and gravy; level.", "Sprinkle shredded cheese on top and bake at 350\\u00b0 for approximately 20 minutes or until golden and bubbly."]',
  'diet_type': 'none',
  'allergy_type': 'none'},
 {'title': 'Reeses Cups(Candy)  ',
  'ingredients': '["peanut butter", "graham cracker crumbs", "butter", "powdered sugar", "chocolate chips"]',
  'directions': '["Combine first four ingredients and press in 13 x 9-inch ungreased pan.", "Melt chocolate chips and spread over mixture. Refrigerate for about 20 minutes and cut into pieces before chocolate gets hard.", "Keep in refrigerator."]',
  'diet_type': 'vegetarian',
  'allergy_type': 'peanuts, dairy'}]

In [ ]:
# recipe generator with diet type and allergens
class GPT3RecipeGeneratorDiet:
    def __init__(self, api_key, model_engine="text-davinci-003"):
        self.model_engine = model_engine
        openai.api_key = api_key
        
    def generate_recipe(self, ingredients, diet_type=None, allergy_type=None):
        prompt = f"Generate a recipe that only contains {', '.join(ingredients)}"
        if diet_type:
            prompt += f" and is suitable for {', '.join(diet_type)} diet"
        if allergy_type:
            prompt += f" and is free of {', '.join(allergy_type)} allergens"
        prompt += ":"
        response = openai.Completion.create(
            engine=self.model_engine,
            prompt=prompt,
            max_tokens=2048,
            n=1,
            stop=None,
            temperature=0.8,
            presence_penalty=0.8 # penalize recipe with ingredients not exist in the prompt
        )
        recipe = response["choices"][0]["text"].strip()

        return recipe
        
    def fine_tune(self, fewshot_examples):
        examples = []
        for example in fewshot_examples:
            prompt = f"Recipe: {example['title']}\nIngredients: {', '.join(example['ingredients'])}\n"
            if 'diet_type' in example:
                prompt += f"Diet Type: {', '.join(example['diet_type'])}\n" 
            if 'allergy_type' in example:
                prompt += f"Allergy Type: {', '.join(example['allergy_type'])}\n"
            prompt += f"Directions: {', '.join(example['directions'])}\n\n"
            examples.append(prompt)
        openai.Completion.create(
            model=self.model_engine,
            prompt=examples,
            temperature=0.5,
            max_tokens=1024,
            n=1,
            stop=None,
            presence_penalty=0.8 
        )

In [ ]:
# Create an instance of the GPT3RecipeGenerator class
fewshot_generator_diet = GPT3RecipeGeneratorDiet(OPENAI_API_KEY)

# Fine-tune the model with the few-shot examples
fewshot_generator_diet.fine_tune(new_fewshot_examples_restrictions)

# a generated recipe
recipe = fewshot_generator_diet.generate_recipe(user_input_ingredients, diet_type=user_input_diet_type, allergy_type= user_input_allergens)

RateLimitError: ignored